In [1]:
from xml.etree.ElementPath import prepare_descendant

from networkx.algorithms.shortest_paths.unweighted import predecessor

from AllGraphs import *
from Draw import *
from math import *
import numpy as np

graphConstructor = IncidencyGraph

In [2]:
class Tas :
    def __init__(self, compare = lambda a, b : a - b) :
        self.data = list()
        self.compare = compare

    def __len__(self) :
        # DONE : renvoyer le nombre d'éléments du tas
        return len(self.data)

    def __getitem__(self, i) :
        # DONE : autoriser la notation [i] en lecture
        return self.data[i]

    def __setitem__(self, i, v) :
        # DONE : autoriser la notation [i] en écriture
        self.data[i] = v

    def left_id(self, i) :
        # DONE : renvoyer l'indice du fils gauche
        return 2*i+1

    def right_id(self, i) :
        # DONE : renvoyer l'indice du fils droit
        return 2*i+2

    def parent_id(self, i) :
        # DONE : renvoyer l'indice du parent
        return (i-1)//2

    def tamper_to_root(self, i):
        if i == 0:
            return

        parent_id = self.parent_id(i)

        if self.compare(self[i], self[parent_id]) > 0:
            self[i], self[parent_id] = self[parent_id], self[i]

            if parent_id != 0:
                self.tamper_to_root(parent_id)

    def append(self, v) :
        # DONE : ajouter un élément depuis la fin de la liste et le faire remonter jusqu'à sa position dans l'arbre
        self.data = self.data + [v]
        self.tamper_to_root(len(self.data)-1)
        return self.data

    def tamper_to_leaf(self, i):
        if i >= len(self):
            return

        left_id = self.left_id(i)
        right_id = self.right_id(i)

        if left_id >= len(self):
            return

        best_id = i

        if self.compare(self[left_id], self[best_id]) > 0:
            best_id = left_id

        if right_id < len(self) and self.compare(self[right_id], self[best_id]) > 0:
            best_id = right_id

        if best_id != i:
            self[i], self[best_id] = self[best_id], self[i]
            self.tamper_to_leaf(best_id)

    def pop(self) :
        # DONE : extraire le plus petit élément échanger le dernier élément et le premier élément puis faire descendre le premier élément jusqu'à sa position dans l'arbre

        root = self.data[0]
        self.data[0] = self.data[-1]
        self.data = self.data[:-1]
        self.tamper_to_leaf(0)
        return root

In [3]:
def tri_par_tas(l, compare = lambda a, b : b - a) :
    tas = Tas(compare)
    for e in l :
        tas.append(e)
    r = list()
    while len(tas) > 0 :
        r.append(tas.pop())
    return r

In [4]:
do_full = True
l = [5, 9, 4, 7, 2, 3, 1, 6]
print(tri_par_tas(l))
if do_full :
    from random import *
    l = [randint(-1000, 1000) for i in range(0, 10)]
    print(l)
    r = tri_par_tas(l)
    print(r)
    is_sorted = True
    for i in range(len(r) - 1) :
        if r[i] > r[i + 1] :
            is_sorted = False
    print("Sorted :", is_sorted)

[1, 2, 3, 4, 5, 6, 7, 9]
[-104, 58, 639, -543, 797, -924, 689, 343, -273, -117]
[-924, -543, -273, -117, -104, 58, 343, 639, 689, 797]
Sorted : True


In [5]:
class UnionFind :
    def __init__(self, values) :
        self.parent = {v : v for v in values}
        self.rank = {v : 0 for v in values}

    def find(self, v) :
        # DONE : rechercher le réprésentant de v dans le graphe.
        # DONE : mettre à jour les liens de parenté lors du parcours
        if self.parent[v] != v:
            self.parent[v] = self.find(self.parent[v])
        return self.parent[v]

    def union(self, x, y) :
        # DONE : fusionner les classes de x et y
        xRoot = self.find(x)
        yRoot = self.find(y)

        if xRoot != yRoot:
            if self.rank[xRoot] < self.rank[yRoot]:
                self.parent[xRoot] = yRoot
            else:
                self.parent[yRoot] = xRoot
                if self.rank[xRoot] == self.rank[yRoot]:
                    self.rank[xRoot] += 1

# TODO : une classe Pair comparable pour associer un arc et son poids et l'utiliser via Tas ?


In [6]:
def Kruskal(graph) :
    # DONE : construire l'arbre couvrant minimal par extraction d'arcs minimaux (composants non connectés)
    tas = Tas(compare= lambda a, b: b[2] - a[2])
    connexite = UnionFind(graph.get_vertices())
    couverture = set()

    for u, v in graph.get_edges():
        poids = graph.get_transition(u, v)
        tas.append((u, v, poids))

    while len(tas) > 0:
        u, v = tas.pop()[0:2]
        if connexite.find(u) == connexite.find(v):
            continue
        connexite.union(u, v)
        couverture.add((u, v))
    return couverture


In [7]:
def example_star() :
	graph = graphConstructor()
	graph.add_vertices({"A", "B", "C", "D", "E", "F"})
	edges = {("A", "B") : 1, ("A", "C") : 6, ("A", "F") : 4, ("B", "C") : 5, ("B", "D") : 2, ("B", "E") : 4, ("B", "F") : 5, ("C", "D") : 3, ("D", "E") : 2, ("E", "F") : 6}
	graph.add_edges(edges)
	graph.add_edges({(v, u) : edges[(u, v)] for u, v in edges})
	return graph

In [8]:
print(Kruskal(example_star()))

{('C', 'D'), ('B', 'A'), ('B', 'D'), ('E', 'D'), ('A', 'F')}


In [9]:
def Prim(graph, start) :
    # DONE : construire l'arbre couvrant minimal par diffusion via les arcs minimaux
    tas = Tas(compare= lambda a, b: b[2] - a[2])
    faits = [start]
    couverture = set()

    for v in graph.neighbors(start):
        poids = graph.get_transition(start, v)
        tas.append((start, v, poids))

    while len(faits) < len(graph.get_vertices()):
        u, v = tas.pop()[0:2]

        if v in faits:
            continue

        for w in graph.neighbors(v):
            if w not in faits:
                poids = graph.get_transition(v, w)
                tas.append((v, w, poids))

        faits.append(v)
        couverture.add((u, v))

    return couverture

In [10]:
print(Prim(example_star(), "A"))

{('D', 'C'), ('B', 'D'), ('D', 'E'), ('A', 'F'), ('A', 'B')}


In [18]:
def build_path(adjacences, start, end) :
    # DONE : reconstruire le chemin de start vers end depuis les adjacences données
    if end not in adjacences or adjacences[end] is None:
        return []

    path = [end]
    current = end

    while current != start:
        current = adjacences[current]
        if current is None:
            return []
        path.append(current)

    return path[::-1]

def path_cost(graph, path) :
    # DONE : calculer le coût du chemin
    if not path or len(path) < 2:
        return 0

    cost = 0
    for i in range(len(path) - 1):
        weight = graph.get_transition(path[i], path[i+1])
        if weight is None:
            return np.inf
        cost += weight

    return cost

In [19]:
def Dijkstra(graph, start, end = None) :
    # DONE : implémenter la méthode de recherche de plus court chemin dans un graphe de Dijkstra
    predecessors = {v: None for v in graph.get_vertices()}
    distances = {v: np.inf for v in graph.get_vertices()}
    distances[start] = 0

    queue = set(graph.get_vertices())

    while queue:
        v = min(queue, key= lambda x: distances[x])

        if end is not None and v == end:
            break

        queue.remove(v)

        d = distances[v]

        for n in graph.neighbors(v):
            weight = graph.get_transition(v, n)

            if d + weight < distances[n]:
                distances[n] = d + weight
                predecessors[n] = v

    return predecessors

In [20]:
def example_net() :
	graph = graphConstructor()
	graph.add_vertices({"A", "B", "C", "D", "E", "F"})
	edges = {("A", "B") : 2, ("A", "C") : 5, ("A", "D") : 1, 
	("B", "C") : 3, ("B", "D") : 2, ("C", "D") : 3, ("C", "E") : 1,
	("C", "F") : 5, ("D", "E") : 1, ("E", "F") : 2}
	graph.add_edges(edges)
	graph.add_edges({(v, u) : edges[(u, v)] for u, v in edges})
	return graph

In [21]:
graph = example_net()
adjacences = Dijkstra(graph, "A")
print(adjacences)
path = build_path(adjacences, "A", "F")
print(path)
print(path_cost(graph, path))

{'A': None, 'C': 'E', 'F': 'E', 'E': 'D', 'D': 'A', 'B': 'A'}
['A', 'D', 'E', 'F']
4


In [32]:
def Floyd_Warshall(graph):
    # DONE : implémenter la méthode de recherche de plus court chemin dans un graphe de Floyd-Warshall
    predecessors = {}
    distances = {}
    vertices = list(graph.get_vertices())

    for s in vertices:
        predecessors[s] = {}
        distances[s] = {}
        for t in vertices:
            if s == t:
                distances[s][t] = 0
            elif graph.has_edge(s, t):
                distances[s][t] = graph.get_transition(s, t)
                predecessors[s][t] = s
            else:
                distances[s][t] = np.inf
                predecessors[s][t] = None

    for k in vertices:
        for s in vertices:
            for t in vertices:
                if distances[s][k] + distances[k][t] < distances[s][t]:
                    distances[s][t] = distances[s][k] + distances[k][t]
                    predecessors[s][t] = predecessors[k][t]

    return predecessors


In [33]:
graph = example_net()
adjacences = Floyd_Warshall(graph)
for v in sorted(adjacences.keys()) :
    print(v, adjacences[v])
path = build_path(adjacences["A"], "A", "F")
print(path)
print(path_cost(graph, path))

A {'C': 'E', 'F': 'E', 'E': 'D', 'D': 'A', 'B': 'A'}
B {'A': 'B', 'C': 'B', 'F': 'E', 'E': 'D', 'D': 'B'}
C {'A': 'D', 'F': 'E', 'E': 'C', 'D': 'E', 'B': 'C'}
D {'A': 'D', 'C': 'E', 'F': 'E', 'E': 'D', 'B': 'D'}
E {'A': 'D', 'C': 'E', 'F': 'E', 'D': 'E', 'B': 'D'}
F {'A': 'D', 'C': 'E', 'E': 'F', 'D': 'E', 'B': 'D'}
['A', 'D', 'E', 'F']
4


In [34]:
def example_neg() :
	graph = graphConstructor()
	graph.add_vertices({"A", "B", "C", "D"})
	edges = {("A", "B") : 2, ("A", "D") : 5, ("D", "B") : -5, ("B", "C") : 2, ("D", "C") : -2}
	graph.add_edges(edges)
	return graph

In [35]:
graph = example_neg()
adjacences = Dijkstra(graph, "A")
print(adjacences)
path = build_path(adjacences, "A", "C")
print(path)
print(path_cost(graph, path))

{'A': None, 'B': 'D', 'C': 'D', 'D': 'A'}
['A', 'D', 'C']
3


In [36]:
graph = example_neg()
adjacences = Floyd_Warshall(graph)
for v in sorted(adjacences.keys()) :
    print(v, adjacences[v])
path = build_path(adjacences["A"], "A", "C")
print(path)
print(path_cost(graph, path))

A {'B': 'D', 'C': 'B', 'D': 'A'}
B {'A': None, 'C': 'B', 'D': None}
C {'A': None, 'B': None, 'D': None}
D {'A': None, 'B': 'D', 'C': 'B'}
['A', 'D', 'B', 'C']
2


In [45]:
def Bellman_Ford(graph, start, end = None) :
    # DONE : implémenter la méthode robuste aux arcs de poids négatif de Bellman-Ford
    predecessors = {v: None for v in graph.get_vertices()}
    distances = {v: np.inf for v in graph.get_vertices()}
    distances[start] = 0
    queue = graph.get_vertices()

    for i in range(1, len(graph.get_vertices())):
        for s, t in graph.get_edges():
            d = distances[s] + graph.get_transition(s, t)

            if d < distances[t]:
                distances[t] = d
                predecessors[t] = s

    for s, t in graph.get_edges():
        if distances[s] + graph.get_transition(s, t) < distances[t]:
            return {}

    return predecessors

In [46]:
graph = example_neg()
adjacences = Bellman_Ford(graph, "A")
print(adjacences)
path = build_path(adjacences, "A", "C")
print(path)
print(path_cost(graph, path))

{'A': None, 'B': 'D', 'C': 'B', 'D': 'A'}
['A', 'D', 'B', 'C']
2


In [57]:
def Astar(graph, start, end, heuristic = lambda x : 1) :
    # DONE : implémenter la méthode de recherche de chemin A* guidée par une heuristique
    predecessors = {v: None for v in graph.get_vertices()}
    costs = {v: np.inf for v in graph.get_vertices()}
    costs[start] = 0
    queue = Tas(lambda a, b: (costs[a] + heuristic(a)) - (costs[b] + heuristic(b)))
    queue.append(start)
    done = set()

    while len(queue) > 0:
        v = queue.pop()
        done.add(v)
        if v == end:
            return predecessors

        for n in graph.neighbors(v):
            c = costs[v] + graph.get_transition(v, n) + heuristic(n)
            if c < costs[n]:
                predecessors[n] = v
                costs[n] = c
                queue.append(n)

    return -1

In [62]:
connex = 8
grid = """
.............
......#####..
......#...#..
.S........#.E
......#...#..
......#####..
.............
"""
grid = [l for l in grid.split("\n") if len(l) > 0]
start = None
end = None
graph = graphConstructor()
print("\n".join(grid))
compute_distance = lambda a, b : ((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2) ** 0.5
for i in range(len(grid)) :
    for j in range(len(grid[i])) :
        if grid[i][j] != "#" :
            graph.add_vertex((i, j))
        if grid[i][j] == "S" :
            start = (i, j)
        if grid[i][j] == "E" :
            end = (i, j)
nei = [(1, 0), (0, 1), (-1, 0), (0, -1)]
if connex == 8 :
    nei += [(1, 1), (1, -1), (-1, 1), (-1, -1)]
for i in range(len(grid)) :
    for j in range(len(grid[i])) :
        if grid[i][j] == "#" : continue
        for ox, oy in nei :
            if i + ox >= 0 and i + ox < len(grid) and j + oy >= 0 and j + oy < len(grid[i]) and grid[i + ox][j + oy] != "#" :
                d = (ox ** 2 + oy ** 2) ** 0.5
                graph.add_edge((i, j), (i + ox, j + oy), d)
                graph.add_edge((i + ox, j + oy), (i, j), d)
for choix in ["Dijkstra", "Bellman_Ford", "Astar", "Floyd_Warshall"] :
    print(choix)
    if choix == "Astar" :
        path = build_path(eval(choix)(graph, start, end, heuristic = lambda x : 10 * compute_distance(x, end) ** 10), start, end)
    if choix == "Floyd_Warshall" :
        path = build_path(eval(choix)(graph)[start], start, end)
    else :
        path = build_path(eval(choix)(graph, start, end), start, end)
    print(path)
    distance = 0
    for i in range(len(path) - 1) :
        distance += compute_distance(path[i], path[i + 1])
    print("- distance : %g" % distance)
    path = set(path)
    for i in range(len(grid)) :
        line = ""
        for j in range(len(grid[i])) :
            if (i, j) in path :
                line += "o"
            else :
                line += grid[i][j]
        print(line)

.............
......#####..
......#...#..
.S........#.E
......#...#..
......#####..
.............
Dijkstra
[(3, 1), (3, 2), (3, 3), (2, 4), (1, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (1, 11), (2, 11), (3, 12)]
- distance : 14.0711
......ooooo..
.....o#####o.
....o.#...#o.
.ooo......#.o
......#...#..
......#####..
.............
Bellman_Ford
[(3, 1), (3, 2), (2, 3), (2, 4), (1, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (1, 11), (2, 12), (3, 12)]
- distance : 14.0711
......ooooo..
.....o#####o.
...oo.#...#.o
.oo.......#.o
......#...#..
......#####..
.............
Astar
[(3, 1), (4, 0), (5, 1), (6, 2), (5, 3), (4, 4), (5, 5), (6, 6), (6, 7), (6, 8), (6, 9), (6, 10), (5, 11), (4, 12), (3, 12)]
- distance : 17.7279
.............
......#####..
......#...#..
.o........#.o
o...o.#...#.o
.o.o.o#####o.
..o...ooooo..
Floyd_Warshall
[(3, 1), (3, 2), (3, 3), (4, 4), (5, 5), (6, 6), (6, 7), (6, 8), (6, 9), (6, 10), (5, 11), (4, 11), (3, 12)]
- distance : 14.0711
.............
......#####..
.